In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# Langchain api key for langsmith tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
# Open AI Credentials from .env file
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="SIMPLE GEN AI APPLICATION"

In [3]:
# data ingestion from website

from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI ReferenceMorePeopleContributingTemplatesCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1🦜️🔗LangSmithLangSmith DocsTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generati

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitters=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
splitted_docs=text_splitters.split_documents(documents=docs)

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS

vector_store_db=FAISS.from_documents(splitted_docs,embedding=embeddings)

In [11]:
vector_store_db

In [13]:
query="LangChain is a framework"
result= vector_store_db.similarity_search(query=query)
result[0].page_content

"v0.2astream_events v2ChangesMigrating to LCELHow to migrate chains to LCELMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from RetrievalQASecurityIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM application lifecycle:Development: Build your applications using LangChain's open-source building blocks, components,"

In [15]:
# retrieval, document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """Answer the following Question based only on the provided context:
    <context>
    {context}
    </context>"""
)


In [16]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001E643F8D610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E66050F890>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')

In [18]:
document_chain=create_stuff_documents_chain(llm,prompt)

In [21]:
from langchain_core.documents import Document
document_chain.invoke({"input":"LangChain is a framework",
                       "context":[Document(page_content="""LangChain is a framework for developing applications powered by large language models (LLMs).

LangChain simplifies every stage of the LLM application lifecycle:

Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations. Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your chains, so that you can continuously optimize and deploy with confidence.
Deployment: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Cloud.""")]})

'LangChain is a framework that aids in the development of applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, from development to deployment. For development, LangChain provides open-source building blocks, components, and third-party integrations, and allows the use of LangGraph to build stateful agents. For productionization, LangSmith can be used to inspect, monitor and evaluate chains to optimize and confidently deploy them. Deployment can be done by transforming LangGraph applications into production-ready APIs and Assistants with LangGraph Cloud.'

In [23]:
# Input----->Retriever----->vector store DB

retriever=vector_store_db.as_retriever()
from langchain.chains import create_retrieval_chain

retriever_chain=create_retrieval_chain(retriever,document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E643D8B2D0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer the following Question based only on the provided context:\n    <context>\n    {context}\n    </context>'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001E643F8D610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000

In [24]:
## get response from LLM

response=retriever_chain.invoke({"input":"what is langchain"})
response


{'input': 'what is langchain',
 'context': [Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
  Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="v0.2astream_events v2ChangesMigrating to LCELHow to migrate chains to LCELMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from RetrievalQASecurityIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM

In [26]:
response['answer']

"LangChain is a framework designed for developing applications that are powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, including development. Users can build applications using LangChain's open-source building blocks, components, and third-party integrations. LangSmith helps users trace and evaluate their language model applications and intelligent agents, moving from prototype to production. LangGraph allows users to build stateful, multi-actor applications with LLMs, integrating smoothly with LangChain. LangChain also provides resources on security best practices."

In [27]:
response['context']

[Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/v0.2/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="v0.2astream_events v2ChangesMigrating to LCELHow to migrate chains to LCELMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from RetrievalQASecurityIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by large language models (LLMs).LangChain simplifies every stage of the LLM application lifecycle:Development: Build yo